In [ ]:
!pip install diffusers["torch"] transformers
!pip install accelerate
!pip install imageio
!pip install gradio==3.47.1

In [ ]:
import gradio as gr
import imageio
import torch
import numpy as np
from transformers import pipeline
from diffusers.utils import load_image, make_image_grid
from diffusers import StableDiffusionControlNetInpaintPipeline, ControlNetModel, UniPCMultistepScheduler

controlnet = ControlNetModel.from_pretrained("lllyasviel/control_v11p_sd15_inpaint", torch_dtype=torch.float16, use_safetensors=True)
pipe = StableDiffusionControlNetInpaintPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16, use_safetensors=True)
pipe.scheduler =  UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

# Function from huggingface
# Description - Create a function to prepare the control image from the initial and mask images.
# This’ll create a tensor to mark the pixels in init_image as masked if the corresponding pixel in mask_image is over a certain threshold.
def make_inpaint_condition(image, image_mask):
    image = np.array(image.convert("RGB")).astype(np.float32) / 255.0
    image_mask = np.array(image_mask.convert("L")).astype(np.float32) / 255.0

    assert image.shape[0:1] == image_mask.shape[0:1]
    image[image_mask > 0.5] = -1.0  # set as masked pixel
    image = np.expand_dims(image, 0).transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    return image

def interior_generate(img,txt):
  init_image = img['image']
  imageio.imwrite("init_image.png", init_image)
  mask_image = img['mask']
  imageio.imwrite("image_mask.png", mask_image)

  mask_image = load_image("./image_mask.png")
  init_image = load_image("./init_image.png")

  init_image = init_image.resize((512, 512))
  mask_image = mask_image.resize((512, 512))
  control_image = make_inpaint_condition(init_image, mask_image)

  prompt=txt
  negative_prompt = "(((Ugly))), windows, doors, dining table, curtains, false ceilings, low-resolution, morbid, blurry, cropped, deformed, dehydrated, text, disfigured, duplicate, error, extra arms, extra fingers, extra legs, extra limbs, fused fingers, gross proportions, jpeg artifacts, long neck, low resolution, tiling, poorly drawn feet, extra limbs, disfigured, body out of frame, cut off, low contrast, underexposed, overexposed, bad art, beginner, amateur, distorted face, low quality, lowres, low saturation, deformed body features, watermark, water mark"
  output = pipe(prompt, negative_prompt=negative_prompt, image=init_image, mask_image=mask_image, control_image=control_image, strength=1, guidance_scale=7.5, num_inference_steps=100).images[0]
  return init_image,output

with gr.Blocks() as demo:
  with gr.Column():
    img = gr.Image(tool="sketch", label="base image", show_label=True, height=512)
    txt = gr.Textbox(label="prompt", lines=2)
    with gr.Row():
      img1 = gr.Image(label="Original image", show_label=True, height=512)
      img2 = gr.Image(label="Generated image", show_label=True, height=512)
  btn = gr.Button()
  btn.click(interior_generate, [img,txt], [img1,img2])

demo.launch(debug=True)